In [ ]:
import numpy as np
import random
import time
import heapq

# Static Maze Definition
static_maze = [
    ['', '', '', '', '', '', '', '', '', ''],
    ['|', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '|'],
    ['|', ' ', '|', '|', ' ', '|', '|', '|', ' ', '|'],
    ['|', ' ', ' ', ' ', ' ', '|', ' ', ' ', ' ', '|'],
    ['|', '|', '|', ' ', '|', ' ', '|', '|', '|', '|'],
    ['|', ' ', ' ', ' ', '|', ' ', ' ', ' ', ' ', '|'],
    ['|', ' ', '|', '|', '|', '|', '|', '|', '|', '|'],
    ['|', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '|'],
    ['|', '|', ' ', '|', '|', '|', '|', '|', '|', '|'],
    ['|', ' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ', '|'],
    ['|', '', '', '', '', '', '', '', '', '|'],
]


# Generate Random Maze
def generate_maze(width, height):
    random.seed(time.time())
    maze = np.ones((height, width), dtype=int)

    def carve_passages(x, y):
        directions = [(x + 2, y), (x - 2, y), (x, y + 2), (x, y - 2)]
        random.shuffle(directions)
        for nx, ny in directions:
            if 0 <= nx < width and 0 <= ny < height and maze[ny][nx] == 1:
                maze[y][x] = 0
                maze[ny][nx] = 0
                maze[(y + ny) // 2][(x + nx) // 2] = 0
                carve_passages(nx, ny)

    carve_passages(random.randrange(1, width, 2), random.randrange(1, height, 2))
    return maze


# A* Algorithm
def a_star(maze, start, end):
    rows, cols = len(maze), len(maze[0])

    def heuristic(a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    priority_queue = []
    heapq.heappush(priority_queue, (0, start))
    came_from = {}
    cost = {start: 0}
    total_cost = {start: heuristic(start, end)}

    while priority_queue:
        current = heapq.heappop(priority_queue)[1]
        if current == end:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]

        i, j = current
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

        for dx, dy in directions:
            neighbor = (i + dx, j + dy)
            ni, nj = neighbor
            if 0 <= ni < rows and 0 <= nj < cols and maze[ni][nj] in (" ", 0):
                currentBestCost = cost[current] + 1
                if currentBestCost < cost.get(neighbor, float('inf')):
                    came_from[neighbor] = current
                    cost[neighbor] = currentBestCost
                    total_cost[neighbor] = currentBestCost + heuristic(neighbor, end)
                    heapq.heappush(priority_queue, (total_cost[neighbor], neighbor))

    return None


# Beam Search Algorithm
def beam_search(maze, start, end, beam_width=3):
    rows, cols = len(maze), len(maze[0])

    def heuristic(a, b):
        return abs(a[0] - b[0]) + abs(a[1] - b[1])

    # List of states at each level (level 0 is the start)
    current_level = [(start, [start])]
    explored = set()

    while current_level:
        next_level = []
        for state, path in current_level:
            i, j = state

            if state == end:
                return path


            directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
            neighbors = []
            for dx, dy in directions:
                ni, nj = i + dx, j + dy
                if 0 <= ni < rows and 0 <= nj < cols and maze[ni][nj] in (" ", 0):
                    if (ni, nj) not in explored:
                        neighbors.append(((ni, nj), path + [(ni, nj)]))

            # Sort neighbors by heuristic (total_cost) and keep only the top beam_width neighbors
            neighbors.sort(key=lambda x: heuristic(x[0], end))
            next_level.extend(neighbors[:beam_width])

        # Mark visited nodes at this level
        for state, _ in next_level:
            explored.add(state)

        # Move to next level
        current_level = next_level

    return None


# Visualize Maze as Grid
def visualize_maze_grid(maze, a_star_path=None, beam_search_path=None, start=None, end=None, is_static_maze=False):
    grid = []
    for i, row in enumerate(maze):
        grid_row = []
        for j, cell in enumerate(row):
            if (i, j) == start:
                grid_row.append("S")
            elif (i, j) == end:
                grid_row.append("E")
            elif a_star_path and (i, j) in a_star_path:
                grid_row.append("*")
            elif beam_search_path and (i, j) in beam_search_path:
                grid_row.append("+")
            else:
                if is_static_maze:
                    grid_row.append("|" if cell == "|" else ".")  # Static maze uses '|'
                else:
                    grid_row.append("!" if cell == "!" else " ")  # Random maze uses '#'
        grid.append(grid_row)
    return grid


def print_grid(grid):
    for row in grid:
        print(" ".join(row))


# Randomly choose static or random maze
if random.choice([True, False]):
    print("---------------------Using static maze---------------------\n\n")
    maze = static_maze
    start, end = (1, 1), (9, 8)
    is_static_maze = True
else:
    print("---------------------Using random maze---------------------\n\n")
    maze = generate_maze(11, 11)
    maze = [['!' if cell == 1 else ' ' for cell in row] for row in maze]
    start, end = (1, 1), (9, 9)
    is_static_maze = False


# Separate calls for A* and Beam Search

# A* Pathfinding
print("Running A* Algorithm:")
a_star_path = a_star(maze, start, end)
grid_a_star = visualize_maze_grid(maze, a_star_path=a_star_path, start=start, end=end, is_static_maze=is_static_maze)
print("A* Pathfinding Result:\n")
print_grid(grid_a_star)
print("\n")

# Beam Search Pathfinding
print("\nRunning Beam Search Algorithm:")
beam_search_path = beam_search(maze, start, end, beam_width=3)
grid_beam_search = visualize_maze_grid(maze, beam_search_path=beam_search_path, start=start, end=end, is_static_maze=is_static_maze)
print("Beam Search Pathfinding Result:\n")
print_grid(grid_beam_search)